In [161]:
#import api data
import requests
url= 'https://api.covidactnow.org/v2/counties.json?apiKey=6abf0fcadd5d4ddab5f43d619f68871e'
resp = requests.get(url=url).json()

In [162]:
#check responce
resp[4]

{'fips': '02060',
 'country': 'US',
 'state': 'AK',
 'county': 'Bristol Bay Borough',
 'hsa': '820',
 'hsaName': 'Anchorage (Anchorage), AK - Matanuska-Susitna Borough, AK',
 'level': 'county',
 'lat': None,
 'locationId': 'iso1:us#iso2:us-ak#fips:02060',
 'long': None,
 'population': 836,
 'hsaPopulation': 547697,
 'metrics': {'testPositivityRatio': None,
  'testPositivityRatioDetails': {'source': 'other'},
  'caseDensity': 0.0,
  'weeklyNewCasesPer100k': 0.0,
  'contactTracerCapacityRatio': None,
  'infectionRate': 1.0,
  'infectionRateCI90': 1.17,
  'icuCapacityRatio': None,
  'bedsWithCovidPatientsRatio': 0.02,
  'weeklyCovidAdmissionsPer100k': 3.3,
  'vaccinationsInitiatedRatio': None,
  'vaccinationsCompletedRatio': None,
  'vaccinationsAdditionalDoseRatio': None},
 'riskLevels': {'overall': 0,
  'testPositivityRatio': 4,
  'caseDensity': 0,
  'contactTracerCapacityRatio': 4,
  'infectionRate': 1,
  'icuCapacityRatio': 4},
 'cdcTransmissionLevel': 0,
 'communityLevels': {'cdcComm

In [124]:
#clean data
def removeele(data,key):
    value = data[key]
    fips_c = data['fips']
    state = data['state']
    coun = data['county']
    data.clear()
    data['fips'] = fips_c
    data[key] = value

In [23]:
#get fips and risk from the api
for i in resp:
     val = i['riskLevels']['overall']
     fips_c = i['fips']
     i.clear()
     i['fips'] = fips_c
     i['risk'] = val

In [28]:
dictresp ={}
dictresp['data'] = resp

In [67]:
#risk json file
import json
with open('risk.json', 'w') as f:
    json.dump(dictresp, f)

In [126]:
#import health data
dataH = json.load(open('dataMapHealth.json'))
dataH = dataH['features']

In [125]:
dataH

[{'type': 'Feature',
  'properties': {'Staffed All Beds': 55.0,
   'Staffed ICU Beds': 6.0,
   'Licensed All Beds': 85.0,
   'All Bed Occupancy Rate': 0.62,
   'ICU Bed Occupancy Rate': 0.53,
   'Population': 55601.0,
   'Population (20+)': 41177.0,
   'Population (65+)': 8653.0,
   'Staffed All Beds [Per 1000 People]': 0.989,
   'Staffed All Beds [Per 1000 Adults (20+)]': 1.336,
   'Staffed All Beds [Per 1000 Elderly (65+)]': 6.356,
   'Staffed ICU Beds [Per 1000 People]': 0.108,
   'Staffed ICU Beds [Per 1000 Adults (20+)]': 0.146,
   'Staffed ICU Beds [Per 1000 Elderly (65+)]': 0.693,
   'Licensed All Beds [Per 1000 People]': 1.529,
   'Licensed All Beds [Per 1000 Adults (20+)]': 2.064,
   'Licensed All Beds [Per 1000 Elderly (65+)]': 9.823,
   'fips': '01001'},
  'geometry': {'type': 'Polygon',
   'coordinates': [[[-86.496774, 32.344437],
     [-86.717897, 32.402814],
     [-86.814912, 32.340803],
     [-86.890581, 32.502974],
     [-86.917595, 32.664169],
     [-86.71339, 32.66173

In [127]:
resultArray = []
for i in dataH:
    my_dict = i['properties']
    del my_dict['State']
    del my_dict['County Name']
    del my_dict['ICU Bed Source']
    del my_dict['ICU Bed Source Last Updated']
    my_dict['fips'] = my_dict.pop('fips_code')
    data = i['properties']
    resultArray.append(data)
    
    

In [128]:
resultArray

[{'Staffed All Beds': 55.0,
  'Staffed ICU Beds': 6.0,
  'Licensed All Beds': 85.0,
  'All Bed Occupancy Rate': 0.62,
  'ICU Bed Occupancy Rate': 0.53,
  'Population': 55601.0,
  'Population (20+)': 41177.0,
  'Population (65+)': 8653.0,
  'Staffed All Beds [Per 1000 People]': 0.989,
  'Staffed All Beds [Per 1000 Adults (20+)]': 1.336,
  'Staffed All Beds [Per 1000 Elderly (65+)]': 6.356,
  'Staffed ICU Beds [Per 1000 People]': 0.108,
  'Staffed ICU Beds [Per 1000 Adults (20+)]': 0.146,
  'Staffed ICU Beds [Per 1000 Elderly (65+)]': 0.693,
  'Licensed All Beds [Per 1000 People]': 1.529,
  'Licensed All Beds [Per 1000 Adults (20+)]': 2.064,
  'Licensed All Beds [Per 1000 Elderly (65+)]': 9.823,
  'fips': '01001'},
 {'Staffed All Beds': 25.0,
  'Staffed ICU Beds': 6.0,
  'Licensed All Beds': 25.0,
  'All Bed Occupancy Rate': 0.5,
  'ICU Bed Occupancy Rate': 0.34,
  'Population': 57840.0,
  'Population (20+)': 43175.0,
  'Population (65+)': 10548.0,
  'Staffed All Beds [Per 1000 People]':

In [129]:
dictresp ={}
dictresp['data'] = resultArray

In [130]:
#hospital json file
import json
with open('hospital2.json', 'w') as f:
    json.dump(dictresp, f)

In [165]:
#Step 3 join risk and hospital data
#import health data
dataR = json.load(open('risk.json'))
dataR = dataR['data']
#import health data
dataH = json.load(open('hospital2.json'))
dataH = dataH['data']

In [132]:
import pandas as pd
dr = pd.DataFrame(dataR)
dh = pd.DataFrame(dataH)

In [158]:
dataR

[{'fips': '02013', 'risk': 2},
 {'fips': '02016', 'risk': 1},
 {'fips': '02020', 'risk': 3},
 {'fips': '02050', 'risk': 3},
 {'fips': '02060', 'risk': 0},
 {'fips': '02068', 'risk': 1},
 {'fips': '02070', 'risk': 5},
 {'fips': '02090', 'risk': 2},
 {'fips': '02100', 'risk': 2},
 {'fips': '02105', 'risk': 0},
 {'fips': '02110', 'risk': 3},
 {'fips': '02122', 'risk': 2},
 {'fips': '02130', 'risk': 2},
 {'fips': '02150', 'risk': 2},
 {'fips': '02158', 'risk': 3},
 {'fips': '02164', 'risk': 0},
 {'fips': '02170', 'risk': 2},
 {'fips': '02180', 'risk': 5},
 {'fips': '02185', 'risk': 2},
 {'fips': '02188', 'risk': 3},
 {'fips': '02195', 'risk': 3},
 {'fips': '02198', 'risk': 1},
 {'fips': '02220', 'risk': 5},
 {'fips': '02230', 'risk': 3},
 {'fips': '02240', 'risk': 2},
 {'fips': '02261', 'risk': 2},
 {'fips': '02275', 'risk': 3},
 {'fips': '02282', 'risk': 0},
 {'fips': '02290', 'risk': 2},
 {'fips': '01001', 'risk': 1},
 {'fips': '01003', 'risk': 1},
 {'fips': '01005', 'risk': 0},
 {'fips'

In [166]:
#combine both the hospital and risk data in one json file and remove uncommon fib elements
count = 0
for i in dataR:
    fips = i['fips']
    exist = False
    for j in dataH:
        if j['fips'] == fips:
            j['risk'] = i['risk']
            exist = True
            count += 1
    if not exist:
        dataH[count]['risk'] = 0

In [168]:
dictresp ={}
dictresp['data'] = resp

In [169]:
#hospital json file
import json
with open('hospitalandrisk.json', 'w') as f:
    json.dump(dictresp, f)

In [170]:
#STEP 4 create hospital index

In [182]:
#risk json file
dataF = json.load(open('facilityMap.json'))

JSONDecodeError: Expecting ',' delimiter: line 4539 column 30 (char 144817)

In [176]:
dataF = dataF['features']

In [ ]:
resultArray = []
for i in dataF:
    my_dict = i['properties']
    del my_dict['State']
    del my_dict['County Name']
    del my_dict['ICU Bed Source']
    del my_dict['ICU Bed Source Last Updated']
    my_dict['fips'] = my_dict.pop('fips_code')
    data = i['properties']
    resultArray.append(data)
    